In [1]:
from gnnboundary.utils.boundary_baseline import BaselineGenerator
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np

from gnnboundary import *

In [3]:
seed = 12345
dataset_name = "motif"

datasets = {
 "collab": CollabDataset(seed=seed),
 "motif": MotifDataset(seed=seed),
 "enzymes": ENZYMESDataset(seed=seed)
}

dataset = datasets[dataset_name]

In [7]:
config = {
    "node_features": len(dataset.NODE_CLS),
    "num_classes": len(dataset.GRAPH_CLS),
}

match dataset_name:
    case "collab":
        config["hidden_channels"] = 64
        config["num_layers"] = 5
    case "motif":
        config["hidden_channels"] = 6
        config["num_layers"] = 3
    case "enzymes":
        config["hidden_channels"] = 32
        config["num_layers"] = 3
    case _:
        pass

model = GCNClassifier(**config)
model.load_state_dict(torch.load(f"ckpts/{dataset_name}.pt"))

<All keys matched successfully>

In [8]:
dataset_list_gt = dataset.split_by_class()
dataset_list_pred = dataset.split_by_pred(model)

C:\Users\40774\miniconda3\envs\gnnboundary\Lib\site-packages\torch_geometric\data\in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The data of the dataset is already cached, so any modifications to `data` will not be reflected when accessing its elements. Clearing the cache now by removing all elements in `dataset._data_list`. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [54]:
from gnnboundary.utils import BaselineGenerator
generator = BaselineGenerator(dataset_list_gt, [0, 1])
samples = generator.sample(500)

In [55]:
model.eval()
model.forward(dataset.convert(samples))

{'logits': tensor([[  2.6943,  -2.4076,   0.8165, -16.9743],
         [ -1.6338,  12.3863, -26.5659,   6.5137],
         [ -1.6364,  11.8628, -25.6314,   5.9107],
         [ -6.5746,   9.8233, -31.7823,   5.2988],
         [ -4.0561,   5.7960, -22.6268,  -2.1732]], grad_fn=<AddmmBackward0>),
 'probs': tensor([[8.6280e-01, 5.2505e-03, 1.3195e-01, 2.4771e-09],
         [8.1274e-07, 9.9719e-01, 1.2080e-17, 2.8077e-03],
         [1.3684e-06, 9.9740e-01, 5.1919e-17, 2.5936e-03],
         [7.4784e-08, 9.8928e-01, 8.4382e-19, 1.0724e-02],
         [5.2616e-05, 9.9960e-01, 4.5287e-13, 3.4584e-04]],
        grad_fn=<SoftmaxBackward0>),
 'embeds': tensor([[-7.9656e+01,  4.7470e+01,  3.6517e+00, -8.8096e+00, -1.7062e+00,
          -2.1349e+00, -8.4740e-01,  5.0501e-01,  3.8848e-02, -9.3719e-02,
          -1.8151e-02, -2.2711e-02],
         [-4.6818e+01,  3.0863e+01,  1.4830e+00, -7.3004e+00, -3.0632e+00,
          -1.0982e+00, -7.6750e-01,  5.0595e-01,  2.4312e-02, -1.1968e-01,
          -5.0217e